# Guided Backpropagation

### 1. Import Dependencies

In [ ]:
import os

from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import nn_ops, gen_nn_ops
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# from models_3_3 import MNIST_CNN
from models_6_1 import MNIST_CNN

%matplotlib inline

mnist = input_data.read_data_sets("data/fashion", one_hot=True)

images = mnist.train.images
labels = mnist.train.labels

logdir = './tf_logs/3_3_GBP/'
ckptdir = logdir + 'model'

if not os.path.exists(logdir):
    os.mkdir(logdir)

In [ ]:
# https://stackoverflow.com/questions/38340791/guided-back-propagation-in-tensorflow

grad = tf.placeholder(tf.float32, [2,2])
feat = tf.placeholder(tf.float32, [2,2])

# Forward pass
frwd = tf.nn.relu(feat)

# Gradient calculation using backpropagation
res1 = gen_nn_ops._relu_grad(grad, feat)

# Gradient calculation using deconvolution
res2 = tf.nn.relu(grad)

# Gradient calculation using guided backpropagation
res3 = tf.where(0. < grad, gen_nn_ops._relu_grad(grad, feat), tf.zeros(grad.get_shape()))

sess = tf.InteractiveSession()

grad_inp = np.array([[-1, -1], [1, 1]])
feat_inp = np.array([[-2, 2], [2, -2]])

f, r1, r2, r3 = sess.run([frwd, res1, res2, res3], feed_dict={grad: grad_inp, feat: feat_inp})

print('Feature:\n', feat_inp)
print('Gradient:\n', grad_inp)
print('Forward Pass:\n', f)
print('Backpropagation:\n', r1)
print('Deconvolution:\n', r2)
print('Guided Backpropagation:\n', r3)

sess.close()

### 2. Building Graph

In [ ]:
with tf.name_scope('Classifier'):
        
    # Initialize neural network
    DNN = MNIST_CNN('CNN')

    # Setup training process
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    Y = tf.placeholder(tf.float32, [None, 10], name='Y')

    activations, logits = DNN(X)
    
    tf.add_to_collection('BP', X)
    tf.add_to_collection('BP', logits)
    
    for activation in activations:
        tf.add_to_collection('BP', activation)

    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

    optimizer = tf.train.AdamOptimizer().minimize(cost, var_list=DNN.vars)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

cost_summary = tf.summary.scalar('Cost', cost)
accuray_summary = tf.summary.scalar('Accuracy', accuracy)
summary = tf.summary.merge_all()

### 3. Training Network

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

# Hyper parameters
training_epochs = 5
batch_size = 1000

for epoch in range(training_epochs):
    total_batch = int(mnist.train.num_examples / batch_size)
    avg_cost = 0
    avg_acc = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, c, a, summary_str = sess.run([optimizer, cost, accuracy, summary], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c / total_batch
        avg_acc += a / total_batch
        
        file_writer.add_summary(summary_str, epoch * total_batch + i)

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost), 'accuracy =', '{:.9f}'.format(avg_acc))
    
    saver.save(sess, ckptdir)

print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

sess.close()

### 4. Restoring Graph

In [ ]:
tf.reset_default_graph()

@ops.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros(tf.shape(grad)))

sess = tf.InteractiveSession()

g = tf.get_default_graph()
with g.gradient_override_map({'Relu': 'GuidedRelu'}):
    new_saver = tf.train.import_meta_graph(ckptdir + '.meta')

new_saver.restore(sess, tf.train.latest_checkpoint(logdir))

activations = tf.get_collection('BP')
print(len(activations))
for i in range(len(activations)):
    print("Activations : \n")
    print activations[i]
X = activations[0]
logits = activations[8]

hmaps = [tf.gradients(logits[:,:,:,i], X)[0] for i in range(10)]

sample_imgs = [images[np.argmax(labels, axis=1) == i][5] for i in range(10)]

### 5. Displaying Images

In [ ]:
features = [sess.run(hmaps[i], feed_dict={X: sample_imgs[i][None]}) for i in range(10)]

plt.figure(figsize=(15,15))
for i in range(5):
    plt.subplot(5, 2, 2 * i + 1)
    plt.imshow(np.reshape(features[2 * i], [28, 28]), cmap='gray', interpolation='none')
    plt.title('Garment: {}'.format(2 * i))
    plt.xticks([])
    plt.yticks([])
    plt.colorbar()
    
    plt.subplot(5, 2, 2 * i + 2)
    plt.imshow(np.reshape(features[2 * i + 1], [28, 28]), cmap='gray', interpolation='none')
    plt.title('Garment: {}'.format(2 * i + 1))
    plt.xticks([])
    plt.yticks([])
    plt.colorbar()

plt.tight_layout()